In [1]:
!pip install pymystem3
!pip install pymorphy2

In [2]:
!pip install razdel

In [3]:
!pip install nltk

In [4]:
import numpy as np 
import pandas as pd
import re
from pymystem3 import Mystem
import pymorphy2
from pymorphy2.tokenizers import simple_word_tokenize

In [5]:
train = pd.read_csv('train.csv', sep=';')
test = pd.read_csv('test.csv', sep=';')

In [6]:
train.head(10)

,id,text,label
0,0,Я не знаю какая Ты в жизни в постели в быту...,1
1,1,Херсон рулит есть д для жизни я п 32 068 160 9...,1
2,2,"решил поглумиться, 15000 итераций 0.001 лр",2
3,3,М.48 ДЛЯ ЛЮДЫ! СМС 093.087.16.85,1
4,4,М 33 позн с д/ж 093 76 46 933 Киев,1
5,5,парень скучает девушки приселайте прилольние м...,1
6,6,"argparse прекрасно делает все то, что мне от н...",0
7,7,Напиши в поиске винды «Переименовать компьютер».,0
8,8,"да и все остальные ""сглаживаются"" спомощью pri...",2
9,9,"вангую, что выйдет чуть больше 1700 в месяц чи...",2


In [7]:
train_text = train['text']

In [8]:
for i in range(len(train_text)):
    train_text[i] = re.sub('[^а-яА-Яa-zA-Z]+', ' ', train_text[i]).lower()

<ipython-input-8-ad61826ae761>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_text[i] = re.sub('[^а-яА-Яa-zA-Z]+', ' ', train_text[i]).lower()


In [9]:
train_text

0        я не знаю какая ты в жизни в постели в быту но...
1                       херсон рулит есть д для жизни я п 
2                            решил поглумиться итераций лр
3                                          м для люды смс 
4                                        м позн с д ж киев
                               ...                        
78721    у меня наверно говенные авиалинии nно вот все ...
78722    он по крайне мере бесплатный имеет нормальный ...
78723                         я уточню у грефа ладно ладно
78724    да пока так и делаю но у ребят все быстро меня...
78725    нее так как я не умею выбирать пусть у меня бу...
Name: text, Length: 78726, dtype: object

In [10]:
%matplotlib inline

import tqdm
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

In [11]:
from sklearn.model_selection import train_test_split

In [48]:
X_train, X_test, y_train, y_test = train_test_split(train['text'], train['label'],
                                                    test_size=0.05)

In [49]:
def evaluate_vectorizer(vectorizer, train, test, y_train, y_test):
    train_vectors = vectorizer.fit_transform(X_train)
    test_vectors = vectorizer.transform(X_test)
    
    clf = LinearSVC(random_state=42)
    clf.fit(train_vectors, y_train)
    
    predictions = clf.predict(test_vectors)
    print(classification_report(y_test, predictions))
    return clf

In [50]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [51]:
X_train

62670                 где бумбокс летний дождь спать хочу 
66733    ну если ты например выкинешь результаты то и н...
77119                парен познак муж л днепр смр смр звн 
12083     м о ищу женщхуденьк поряд не курящ с семьи дн...
45741                     для вашего же удобства исправил 
                               ...                        
71697    познак со свободн стройн дев рыбы для общ и се...
60641    там где то на этот график еще пару работ докид...
22796                             или я что то не понимаю 
42327                   хорошо что не тратила на это время
32768                        м позн с женщ которая всегда 
Name: text, Length: 74789, dtype: object

In [16]:
clf1 = evaluate_vectorizer(TfidfVectorizer(), X_train, X_test, y_train, y_test)

In [17]:
X_train

24381    рыжая бестия встретится сейчас с мужчиной заед...
497      добрый и нежный п л ищу дев л для общ и отноше...
2092     короче если на учете d стоит то вообще каждые ...
78031    зая люблю тока тебя извени тока тебя извени чт...
3514        п скрасит досуг дев жен приеду есть мтс киевст
                               ...                        
28450    п познакомлюсь с девушкой для серьезных отноше...
12766    а многие забили на этот чат и с радостью заним...
33966                        http doc qt io qt layout html
35041             пикл быстрее жсона и умеет сохранять все
2336             если бы сиськи можно было потрогать сразу
Name: text, Length: 55108, dtype: object

In [18]:
import razdel

In [52]:
def tokenize_with_razdel(text):
    tokens = [token.text for token in razdel.tokenize(text)]
    
    return tokens

In [33]:
evaluate_vectorizer(TfidfVectorizer(tokenizer=tokenize_with_razdel), X_train, X_test, y_train, y_test);

In [53]:
from pymorphy2 import MorphAnalyzer

pymorphy = MorphAnalyzer()


def lemmatize_with_pymorphy(tokens):
    lemms = [pymorphy.parse(token)[0].normal_form for token in tokens]
    return lemms

In [61]:
tfidf_vectorizer = TfidfVectorizer(
    tokenizer=lambda text: lemmatize_with_pymorphy(tokenize_with_razdel(text)),
)

clf2 = evaluate_vectorizer(tfidf_vectorizer, X_train, X_test, y_train, y_test)

C:\Users\artem\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['большой', 'весь', 'всё', 'ещё', 'мочь', 'нибыть', 'свой', 'хороший', 'это'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


              precision    recall  f1-score   support

           0       0.79      0.70      0.74       751
           1       0.98      0.97      0.97      1787
           2       0.83      0.88      0.85      1399

    accuracy                           0.89      3937
   macro avg       0.87      0.85      0.86      3937
weighted avg       0.89      0.89      0.89      3937



In [36]:
test_text = test['text']

In [37]:
for i in range(len(test_text)):
    test_text[i] = re.sub('[^а-яА-Яa-zA-Z]+', ' ', test_text[i]).lower()

<ipython-input-37-9d92444a5416>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_text[i] = re.sub('[^а-яА-Яa-zA-Z]+', ' ', test_text[i]).lower()


In [55]:
def get_csv(vectorizer, test, clf):
    test_vectors = vectorizer.transform(test['text'])
    
    target = 'label'
    test[target] = clf.predict(test_vectors)
    test[['id', target]].to_csv('sub_baseline2.csv', index=None)
#     print('Size of vocabulary', len(vectorizer.vocabulary_))

In [62]:
get_csv(tfidf_vectorizer, test, clf2)